### Install Libraries

In [ ]:
!pip install pypdf
!pip install chromadb
!pip install langchain-chroma
!pip install langchain-community
!pip install sentence-transformers

### Import Libraries

In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings.sentence_transformer import (SentenceTransformerEmbeddings,)
from langchain_text_splitters import RecursiveCharacterTextSplitter

### Load the Document

In [ ]:
loader = PyPDFLoader("rag_data.pdf")
documents = loader.load()

### Split into chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

### Create Embeddings

In [ ]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

### Load into Chroma

In [ ]:
db = Chroma.from_documents(docs, embedding_function)

### Query the Database

In [ ]:
query = "explain the background for LLMs?"
docs = db.similarity_search(query)

### Get Relevant Chunks

In [ ]:
print(docs[0].page_content)

**Module 3**

**Importing Libraries**

In [ ]:
!pip install langchain-openai
!pip install langchainhub

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

In [ ]:
OPENAI_API_KEY = "please_provide_your_key_here"

**Provide the OpenAI API key**

In [ ]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

**LLM**

In [ ]:

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")


**Prompt**

In [ ]:

prompt = hub.pull("rlm/rag-prompt")


In [ ]:
prompt

In [ ]:
len(docs)

**Format Docs**

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [ ]:
from langchain.vectorstores import Chroma
retriever = db.as_retriever(search_kwargs={"k": 6})

**RAG Chain**

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("How Emergent Abilities Relate to Scaling Laws?"):
    print(chunk, end="", flush=True)

**Customizing the Prompt**

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking me the question!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("explain the background for LLMs?")

### Congratulations you completed Final Module !!